In [25]:
from docxtpl import DocxTemplate
import os
import pandas as pd
import re

In [26]:
req = {"nodes":[{"id":"v0","type":"violet","data":{"label":"1.  <название>.docx","category":"1. .docx"}},{"id":"v1","type":"violet","data":{"label":"1.1.  <название>.docx","category":"1.1. .docx"}},{"id":"v2","type":"violet","data":{"label":"1.2.  <название>.docx","category":"1.2. .docx"}},{"id":"o1","type":"orange","data":{"label":"разбивать на папки"}},{"id":"00","type":"blue","data":{"label":"О","category":"1. .docx"}},{"id":"01","type":"blue","data":{"label":"Ф","category":"1. .docx"}},{"id":"02","type":"blue","data":{"label":"И","category":"1. .docx"}},{"id":"03","type":"blue","data":{"label":"Номер","category":"1. .docx"}},{"id":"10","type":"blue","data":{"label":"О","category":"1.1. .docx"}},{"id":"11","type":"blue","data":{"label":"Ф","category":"1.1. .docx"}},{"id":"12","type":"blue","data":{"label":"И","category":"1.1. .docx"}},{"id":"13","type":"blue","data":{"label":"ППК","category":"1.1. .docx"}},{"id":"20","type":"blue","data":{"label":"Оценка2","category":"1.2. .docx"}},{"id":"21","type":"blue","data":{"label":"Номер","category":"1.2. .docx"}},{"id":"22","type":"blue","data":{"label":"Оценка1","category":"1.2. .docx"}},{"id":"g2","type":"green","data":{"label":"ФИО"}},{"id":"g3","type":"green","data":{"label":"Фамилия"}},{"id":"g4","type":"green","data":{"label":"Имя"}},{"id":"g5","type":"green","data":{"label":"Отчество"}},{"id":"g6","type":"green","data":{"label":"Номер"}},{"id":"g7","type":"green","data":{"label":"ППК"}},{"id":"g8","type":"green","data":{"label":"Оценка 1"}},{"id":"g9","type":"green","data":{"label":"Оценка 2"}}],"connections":[{"source":"g2","target":"o1"},{"source":"g2","target":"v2"},{"source":"g2","target":"v1"},{"source":"g2","target":"v0"},{"source":"g3","target":"01"},{"source":"g3","target":"11"},{"source":"g5","target":"00"},{"source":"g5","target":"10"},{"source":"g4","target":"02"},{"source":"g4","target":"12"},{"source":"g7","target":"13"},{"source":"g6","target":"21"},{"source":"g8","target":"20"},{"source":"g9","target":"22"}]}

In [27]:
req

{'nodes': [{'id': 'v0',
   'type': 'violet',
   'data': {'label': '1.  <название>.docx', 'category': '1. .docx'}},
  {'id': 'v1',
   'type': 'violet',
   'data': {'label': '1.1.  <название>.docx', 'category': '1.1. .docx'}},
  {'id': 'v2',
   'type': 'violet',
   'data': {'label': '1.2.  <название>.docx', 'category': '1.2. .docx'}},
  {'id': 'o1', 'type': 'orange', 'data': {'label': 'разбивать на папки'}},
  {'id': '00', 'type': 'blue', 'data': {'label': 'О', 'category': '1. .docx'}},
  {'id': '01', 'type': 'blue', 'data': {'label': 'Ф', 'category': '1. .docx'}},
  {'id': '02', 'type': 'blue', 'data': {'label': 'И', 'category': '1. .docx'}},
  {'id': '03',
   'type': 'blue',
   'data': {'label': 'Номер', 'category': '1. .docx'}},
  {'id': '10',
   'type': 'blue',
   'data': {'label': 'О', 'category': '1.1. .docx'}},
  {'id': '11',
   'type': 'blue',
   'data': {'label': 'Ф', 'category': '1.1. .docx'}},
  {'id': '12',
   'type': 'blue',
   'data': {'label': 'И', 'category': '1.1. .docx'

In [28]:
def generate_context(file: str, req: dict) -> dict:
    context = {}
    for node in req["nodes"]:
        if node["type"] == "blue":
            if node['data']['category'] == file:
                for connection in req["connections"]:
                    if connection["target"] == node["id"]:
                        source_node_id = connection["source"]
                        source_node = next(n for n in req["nodes"] if n["id"] == source_node_id)
                        context[node['data']['label']] = source_node['data']['label']
    
    return context


In [29]:
def get_folder_key(req):
    for node in req["nodes"]:
        if node["type"] == "orange":
            for connection in req["connections"]:
                if connection["target"] == node["id"]:
                    source_node_id = connection["source"]
                    source_node = next(n for n in req["nodes"] if n["id"] == source_node_id)
                    return source_node['data']['label']

In [30]:
def get_file_name_key(file: str, req: dict) -> str:
    for node in req["nodes"]:
        if node["type"] == "violet" and node['data']['category'] == file:
            for connection in req["connections"]:
                if connection["target"] == node["id"]:
                    source_node_id = connection["source"]
                    source_node = next(n for n in req["nodes"] if n["id"] == source_node_id)
                    return source_node['data']['label'], node['data']['label']


In [31]:
get_file_name_key('1.1. .docx', req)

('ФИО', '1.1.  <название>.docx')

In [32]:
generate_context('1.1. .docx', req)

{'О': 'Отчество', 'Ф': 'Фамилия', 'И': 'Имя', 'ППК': 'ППК'}

In [38]:
def fill_template_with_docxtpl(row, file, req, zip_dir, folder_key, file_name_key):
    SAVE_DIR = "uploaded_words"
    template_path = os.path.join(SAVE_DIR, file)
    template = DocxTemplate(template_path)

    contex = generate_context(file, req)
    template.render(contex)

    
    folder_name = row[folder_key]
    in_path = os.path.join(zip_dir, folder_name)
    os.makedirs(in_path, exist_ok=True)

    file_name = re.sub(r'<[^<>]*>', file_name_key[0], file_name_key[1])
    save_path = os.path.join(in_path, file_name)
    template.save(save_path)

In [ ]:
def generate_zip(file_names, req):
    SAVE_DIR = "uploaded_words"
    excel_path = os.path.join(SAVE_DIR, "excel.xlsx")
    df = pd.read_excel(excel_path)
    folder_key = get_folder_key(req)

    ZIP_DIR = "generated_zip"
    os.makedirs(ZIP_DIR, exist_ok=True)

    for file_name in file_names:
        file_name_key = get_file_name_key(file_name, req)
        df.apply(fill_template_with_docxtpl, axis=1, file=file_name, req=req, zip_dir = ZIP_DIR, folder_key=folder_key, file_name_key=file_name_key)



In [43]:
def get_file_names(req):
    file_names = []
    for node in req["nodes"]:
        if node["type"] == "violet":
            file_names.append(node['data']['category'])
    return file_names

In [45]:
generate_zip(get_file_names(req), req)

1. .docx
1.1. .docx
1.2. .docx
